DB연결

In [2]:
# !pip install pymysql

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pymysql
import requests
import ssl
context = ssl._create_unverified_context()

1. DB 연결
    - pymysql.connect(host= 'ip주소/도메인/local',port = 3306, user='id',passwd='1234',charset='utf8')
    - db conn 객체 반환

2. cursor 객체 추출(sql 실행 시키는 객체)
    - conn.cursor() 
3. sql 구문 실행
    - cursor.execute(sql)
4. sql 실행 결과 추출(실행된 cursor 객체가 결과를 저장하고 있음)
    - cursor.fetch*()

In [2]:
# 1. db연결 
db = pymysql. connect(host='localhost',port=3306,
                     user='root', passwd='1234',
                     charset='utf8')

In [3]:
# 2. cursor 객체 추출 
cursor = db.cursor()

In [4]:
# 3. sql 실행
sql = 'show databases'
cursor.execute(sql)

10

In [5]:
# 4. 실행 결과 확인
result =cursor.fetchall()
result

(('beauty_shop',),
 ('coffe',),
 ('csv_db2',),
 ('employees',),
 ('information_schema',),
 ('multi',),
 ('mysql',),
 ('performance_schema',),
 ('sqldb',),
 ('test',))

In [6]:
### 사용 DB 생성 
sql = 'create database beauty_shop'
cursor.execute(sql)

ProgrammingError: (1007, "Can't create database 'beauty_shop'; database exists")

In [ ]:
# 생성 확인 
sql = 'show databases'
cursor.execute(sql)
result = cursor.fetchall()
result

In [7]:
# db 선택
sql = 'use beauty_shop'
cursor.execute(sql)

0

In [8]:
# 현재 db 조회 
sql = 'select database()'
cursor.execute(sql)

1

In [9]:
result = cursor.fetchone()
result

('beauty_shop',)

In [10]:
### table 생성  - sql 구문 작성
sql = """
    create table product(
        product_code int auto_increment not null,
        title varchar(200) not null,
        ori_price float,
        discount_price float,
        link varchar(200),
        primary key(product_code)        
        )
"""

In [11]:
cursor.execute(sql)

OperationalError: (1050, "Table 'product' already exists")

In [12]:
db.commit() # db에 반영 (complete)

In [13]:
# 생성 테이블 확인
sql = 'show tables'
cursor.execute(sql)
cursor.fetchall()

(('product',),)

In [14]:
sql = 'desc product' # 테이블 구조 확인 
cursor.execute(sql)
cursor.fetchall()

(('product_code', 'int(11)', 'NO', 'PRI', None, 'auto_increment'),
 ('title', 'varchar(200)', 'NO', '', None, ''),
 ('ori_price', 'float', 'YES', '', None, ''),
 ('discount_price', 'float', 'YES', '', None, ''),
 ('link', 'varchar(200)', 'YES', '', None, ''))

### 제품 정보 추출 후 DB 저장

In [15]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [16]:
# 한개의 제품에 대한 정보 추출
def get_product_info(box) :
    strong_tag = box.find('strong',{'class':'name'})
    prd_name = strong_tag.text.split(':')[1]
    #가격
    price = box.find('ul').findAll('span')[1].text
    sale_price = box.find('ul').findAll('span')[-1].text
    if sale_price == '' :
        sale_price = 'USD 0.0'
    
    return {'품목':prd_name, '가격':price[4:], '세일가격':sale_price[4:]}

In [17]:
# db에 저장하는 함수 
import ssl
context = ssl._create_unverified_context()
def save_data(prd_info) : 
#     print(prd_info)

    sql = "insert into product(title, ori_price, discount_price, link) values('" \
    + prd_info['품목'] + "'," \
    + prd_info['가격'] + "," \
    + prd_info['세일가격'] + ",'" \
    + 'sub_link' + "')"
#     print(sql)
    cursor.execute(sql)

In [18]:
import ssl
context = ssl._create_unverified_context()
def get_page_products(url) :
    url=url
    html = urlopen(url,context=context)
    htmls = html.read()
    bs_obj = BeautifulSoup(htmls,"html.parser")
    boxes = bs_obj.findAll('div',{'class':'description'})
    for box in boxes : 
        prd =get_product_info(box)
        save_data(prd)

In [19]:
url = "http://jolse.com/category/toners-mists/1019/?page=1"
get_page_products(url) 

In [20]:
db.commit()

In [21]:
sql = 'select * from product'
cursor.execute(sql)
result = cursor.fetchall()
result

((1,
  ' [TIME DEAL] Haruharu WONDER Black Rice Hyaluronic Toner 300ml',
  37.0,
  16.59,
  'sub_link'),
 (2,
  ' SOME BY MI Propolis B5 Glow Barrier Calming Toner 150ml',
  22.0,
  18.7,
  'sub_link'),
 (3,
  ' Haruharu WONDER Black Rice Hyaluronic Toner 300ml',
  37.0,
  31.45,
  'sub_link'),
 (4, ' Cos De BAHA Centella Facial Toner 200ml', 16.45, 13.98, 'sub_link'),
 (5, ' Isntree Hyaluronic Acid Toner 200ml (Renewal)', 19.3, 16.4, 'sub_link'),
 (6,
  ' SOME BY MI AHA BHA PHA 30 Days Miracle Toner 150ml',
  24.0,
  20.4,
  'sub_link'),
 (7,
  ' SOME BY MI Galactomyces Pure Vitamin C Glow Toner 200ml',
  34.0,
  28.9,
  'sub_link'),
 (8, ' ROUND LAB 1025 Dokdo Toner 200ml', 17.0, 14.45, 'sub_link'),
 (9,
  ' SOME BY MI Snail Truecica Miracle Repair Toner 135ml',
  24.0,
  20.4,
  'sub_link'),
 (10, ' COSRX AHA 7 WHITEHEAD POWER LIQUID 100ml', 17.81, 15.14, 'sub_link'),
 (11,
  ' BLITHE Vital Treatment 8 Nourishing Beans 150ml',
  42.9,
  36.46,
  'sub_link'),
 (12, ' COSRX Two In One

In [22]:
db.close()

- pandas 함수 이용해서 sql 쿼리 실행

In [23]:
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='1234',db='beauty_shop',  charset='utf8')
cursor = db.cursor()

In [24]:
sql = 'select * from product'
df = pd.read_sql(sql,db)

C:\Users\82107\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [25]:
df

,product_code,title,ori_price,discount_price,link
0,1,[TIME DEAL] Haruharu WONDER Black Rice Hyalur...,37.00,16.59,sub_link
1,2,SOME BY MI Propolis B5 Glow Barrier Calming T...,22.00,18.70,sub_link
2,3,Haruharu WONDER Black Rice Hyaluronic Toner 3...,37.00,31.45,sub_link
3,4,Cos De BAHA Centella Facial Toner 200ml,16.45,13.98,sub_link
4,5,Isntree Hyaluronic Acid Toner 200ml (Renewal),19.30,16.40,sub_link
...,...,...,...,...,...
67,68,COSRX AC Collection Calming Liquid Mild 125ml,26.00,22.10,sub_link
68,69,NACIFIC Real Calendula Floral Toner 180ml,26.00,22.10,sub_link
69,70,CARYOPHY Portulaca Toner 300ml,24.00,20.40,sub_link
70,71,NEOGEN DERMALOGY Real Cica Pad 90ea,17.90,15.21,sub_link


In [26]:
db.close()